In [12]:
import pandas as pd
from itertools import islice
import torch
from torch.utils.data import DataLoader
import sys
sys.path.append(r"C:\Users\emman\Desktop\PROYECTOS_VS_CODE\PRUEBAS_DE_PYTHON\Chest-X-ray-Diagnosis-Automated-Reporting-using-CNNs-and-LLMs---UDEM-PEF-Thesis-Fall-2025")

from utils.text_metrics import evaluate_all_metrics
from utils.temp_utils import *
from utils.gpt_models import DinoGPTCaptioner, DinoGPT2Captioner
from utils.chexpert_dataset import CheXpertDataset
from utils.padchest_dataset import PadChestGRDataset

# Data

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

CSV_PATH = r"C:\Users\emman\Desktop\PROYECTOS_VS_CODE\PRUEBAS_DE_PYTHON\CheXpertPlus\df_chexpert_plus_240401.csv"
IMG_ROOT = r"C:\Users\emman\Desktop\PROYECTOS_VS_CODE\PRUEBAS_DE_PYTHON\CheXpertPlus\PNG"
TEXT_COL = "section_impression"
PATH_COL = "path_to_image"

IMG_SIZE = 224
MAX_LEN = 64
NUM_BATCH = 8

tf = dino_image_transform(img_size=IMG_SIZE)

ds_train = CheXpertDataset(img_root=IMG_ROOT, csv_path=CSV_PATH, split="train", transform=tf, text_col=TEXT_COL)
ds_valid = CheXpertDataset(img_root=IMG_ROOT, csv_path=CSV_PATH, split="valid", transform=tf, text_col=TEXT_COL)
ds_test = CheXpertDataset(img_root=IMG_ROOT, csv_path=CSV_PATH, split="test", transform=tf, text_col=TEXT_COL)

tokenizer = build_tokenizer_from_labels(gpt2=True)
pad_id = tokenizer.pad_token_id
eos_id = tokenizer.eos_token_id
bos_id = tokenizer.bos_token_id
collate_fn = CaptionCollate(tokenizer, pad_id)

train_loader = DataLoader(ds_train, batch_size=NUM_BATCH, shuffle=True, collate_fn=collate_fn)
valid_loader = DataLoader(ds_valid, batch_size=NUM_BATCH, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(ds_test, batch_size=NUM_BATCH, shuffle=False, collate_fn=collate_fn)

Using device: cuda
[INFO] Kept 47494/223462 rows with existing PNGs under C:\Users\emman\Desktop\PROYECTOS_VS_CODE\PRUEBAS_DE_PYTHON\CheXpertPlus\PNG
[INFO] Kept 47494/223462 rows with existing PNGs under C:\Users\emman\Desktop\PROYECTOS_VS_CODE\PRUEBAS_DE_PYTHON\CheXpertPlus\PNG
[INFO] Kept 47494/223462 rows with existing PNGs under C:\Users\emman\Desktop\PROYECTOS_VS_CODE\PRUEBAS_DE_PYTHON\CheXpertPlus\PNG


# Model

In [14]:
# DINO ViT-S/16 hidden size is 384 
EMBEDDING_D_IMG = 384
N_PREFIX = (IMG_SIZE // 16) ** 2  # number of visual prefix tokens (including CLS)

def pick_heads(d_model, target_head_dim=64):
    h = max(1, round(d_model / target_head_dim))
    while d_model % h != 0: h -= 1
    return h

D_MODEL = 768
N_HEAD = pick_heads(D_MODEL, 64)  # -> 12


model = DinoGPT2Captioner(
    d_img=EMBEDDING_D_IMG,
    num_prefix_tokens=N_PREFIX,
    gpt2_name="gpt2",
    dino_model_id="facebook/dinov3-vits16-pretrain-lvd1689m",
    freeze_dino=True
).to(device)

# Print model parameters and trainable parameters
total_params = sum(p.numel() for p in model.parameters())
print(f"Total model parameters: {total_params / 1_000_000:.2f} Millions")

trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Trainable model parameters: {trainable_params / 1_000_000:.2f} Millions")

# Print model footprint
model_footprint_in_gb = (total_params * 4) * (1e-9)  # assuming 4 bytes per parameter (float32)
print(f"Approximate model footprint: {model_footprint_in_gb:.2f} GB")

# after model init
#model.decoder.lm_head.weight = model.decoder.tok_emb.weight  # weight tying

Total model parameters: 146.33 Millions
Trainable model parameters: 124.74 Millions
Approximate model footprint: 0.59 GB


# Train Parameters

In [15]:
optimizer = torch.optim.AdamW(
    filter(lambda p: p.requires_grad, model.parameters()), lr=3e-4, weight_decay=1e-2
)
loss = sequence_ce_loss
NUM_EPOCHS = 100
BATCHES_PER_EPOCH = 10

# Training

In [16]:
for epoch in range(NUM_EPOCHS):
    slice_train_loader = islice(train_loader, BATCHES_PER_EPOCH)
    slice_valid_loader = islice(valid_loader, BATCHES_PER_EPOCH)
    train_stats = train_one_epoch(model, slice_train_loader, optimizer, device, pad_id, num_batches=BATCHES_PER_EPOCH, loss_fn=loss, grad_clip=1.0)
    val_stats = evaluate(model, slice_valid_loader, device, pad_id, num_batches=BATCHES_PER_EPOCH, loss_fn=loss)
    print(f"Epoch {epoch + 1}: Train Loss={train_stats['loss']:.4f}, PPL={train_stats['ppl']:.2f} | "
            f"Val Loss={val_stats['val_loss']:.4f}, Val PPL={val_stats['val_ppl']:.2f}")

Evaluating: 100%|██████████| 10/10 [00:04<00:00,  2.27it/s]


Epoch 1: Train Loss=8.1417, PPL=4012.19 | Val Loss=7.1253, Val PPL=1255.83


Evaluating: 100%|██████████| 10/10 [00:04<00:00,  2.28it/s]


Epoch 2: Train Loss=6.4348, PPL=667.27 | Val Loss=5.6649, Val PPL=294.01


Evaluating: 100%|██████████| 10/10 [00:04<00:00,  2.16it/s]


...

Epoch 98: Train Loss=4.1138, PPL=62.03 | Val Loss=4.0760, Val PPL=59.85


Evaluating: 100%|██████████| 10/10 [00:04<00:00,  2.15it/s]


Epoch 99: Train Loss=4.0858, PPL=60.41 | Val Loss=4.0743, Val PPL=59.69


Evaluating: 100%|██████████| 10/10 [00:04<00:00,  2.12it/s]

Epoch 100: Train Loss=4.0892, PPL=60.26 | Val Loss=4.0762, Val PPL=59.90


# Test Parameters

In [17]:
BATCHES_PER_TEST = 1
GREEDY_DECODE = True
TEST_MAX_LEN = 256
TEST_TOP_P = 0.9
TEST_TEMPERATURE = 0.9

# Test

In [18]:
slice_test_loader = islice(test_loader, BATCHES_PER_TEST)
test_stats = evaluate(model, slice_test_loader, device, pad_id, num_batches=BATCHES_PER_TEST)
print(f"Test Loss={test_stats['val_loss']:.4f}, Test PPL={test_stats['val_ppl']:.2f}")

Evaluating: 100%|██████████| 1/1 [00:00<00:00,  1.90it/s]

Test Loss=4.0206, Test PPL=55.74


# Test Report Generation

In [19]:
with torch.no_grad():
    for pixel_values, ids_loader, paths, raw_labels in test_loader:
        pixel_values = pixel_values.to(device)
        gen_ids = model.generate(
            pixel_values=pixel_values,
            input_ids=ids_loader.to(device),
            max_new_tokens=64
        ).to(device)

        info = model.generate_with_logging(
            pixel_values=pixel_values,
            input_ids=ids_loader.to(device),
            tokenizer=tokenizer,
            preset="safe_sample",
            stop_sequences=["\n\n", "Impression:"],
            max_new_tokens=128,
        )
        print("out shape:", info["sequences"].shape)
        for i, s in enumerate(info["per_sample"]):
            print(f"[{i}] EOS={s['stopping']['hit_eos']} rep={s['repetition']}")
            print(s["text"].get("generated","")[:200])
            print("[Target text]", raw_labels[i])

        eval_results = evaluate_all_metrics(raw_labels, [s["text"]["generated"] for s in info["per_sample"]], evaluation_mode="CheXagent")
        for metric, scores in eval_results.items():
            print(f"{metric}: {scores}")



        print("Predictions (first batch):")
        for i in range(gen_ids.size(0)):
            text_gen = tokenizer.decode(gen_ids[i].tolist())
            text_tgt = tokenizer.decode(ids_loader[i].tolist())
            print(f"\nGEN {i+1}:", text_gen)
            print(f"TGT {i+1}:", text_tgt)
            try: 
                results = evaluate_all_metrics([text_tgt], [text_gen], evaluation_mode="CheXagent")
                for metric, scores in results.items():
                    print(f"{metric}: {scores}")
            except Exception as e:
                print("Error in evaluation:", e)
        del pixel_values, ids_loader, paths, raw_labels, gen_ids
        torch.cuda.empty_cache()
        break

out shape: torch.Size([8, 240])
[0] EOS=False rep={'max_token_run': 1, 'max_repeat_trigram': 1, 'max_repeat_4gram': 1}
cardiomegaly. stable appearance of right chest tube and left chest tube. stable cardiomegtal tube. no acute osseous abnormalities. stable mild pulmonary edema. stable bibasilar opiac opacification and
[Target text] interval placement of a right internal jugular venous sheath with the distal tip in the proximal superior vena cava. no pneumothorax. stable position of nasogastric tube feeding tube tracheostomy canula left internal jugular central venous catheter and left upper extremity picc. no significant interval change in hyperexpanded lung volumes right basilar opacities small bilateral pleural effusions tenting of the right hemidiaphragm and biapical pleural thickening.
[1] EOS=False rep={'max_token_run': 1, 'max_repeat_trigram': 1, 'max_repeat_4gram': 1}
the left internal jugular catheter is unchanged in position. stable positioning. stable appearance of the trache

In [20]:
text = "1.  STABLE SMALL LEFT INTERNAL JUGULAR OPACITIES WITH PATCHY TUBE AND NASOGASTRIC TUBES, RIGHT LOWER MEDIASTINAL SIDED CATHETER.  NO SIGNIFICANT CHANGE IN THE PREVIOUS STUDYDEMONSTRATE ATELECTASIS O"
text = text.lower()
encoded = tokenizer.encode(text)
words = text.split()
print("Number of words:", len(words), "Number of tokens:", len(encoded), "pad_id:", pad_id, "eos_id:", eos_id, "bos_id:", bos_id)
print("BOS token id:", tokenizer.bos_token_id, "EOS token id:", tokenizer.eos_token_id, "PAD token id:", tokenizer.pad_token_id)
print(encoded)
for token_id in encoded:    
    print(f"Token ID: {token_id}, Token: {tokenizer.decode([token_id])}")

Number of words: 27 Number of tokens: 48 pad_id: 50256 eos_id: 50256 bos_id: 50256
BOS token id: 50256 EOS token id: 50256 PAD token id: 50256
[50256, 16, 13, 220, 8245, 1402, 1364, 5387, 45808, 934, 1034, 330, 871, 351, 8529, 88, 12403, 290, 25221, 519, 459, 1173, 21103, 11, 826, 2793, 16957, 459, 1292, 34384, 3797, 43332, 13, 220, 645, 2383, 1487, 287, 262, 2180, 2050, 26567, 23104, 379, 9509, 17765, 267, 50256]
Token ID: 50256, Token: 
Token ID: 16, Token: 1
Token ID: 13, Token: .
Token ID: 220, Token:  
Token ID: 8245, Token:  stable
Token ID: 1402, Token:  small
Token ID: 1364, Token:  left
Token ID: 5387, Token:  internal
Token ID: 45808, Token:  jug
Token ID: 934, Token: ular
Token ID: 1034, Token:  op
Token ID: 330, Token: ac
Token ID: 871, Token: ities
Token ID: 351, Token:  with
Token ID: 8529, Token:  patch
Token ID: 88, Token: y
Token ID: 12403, Token:  tube
Token ID: 290, Token:  and
Token ID: 25221, Token:  nas
Token ID: 519, Token: og
Token ID: 459, Token: ast
Token ID: 

In [21]:
import re
import string

def clean_text(text: str) -> str:
    # lowercase
    text = text.lower()

    # remove enumerators like "1." or "23." but KEEP decimals like "2.5"
    # (?<!\d) ensures no digit right before; (?!\d) ensures no digit right after the dot
    text = re.sub(r'(?<!\d)\b\d+\.(?!\d)', ' ', text)

    # remove all punctuation EXCEPT "."
    punctuation = string.punctuation.replace('.', '')
    text = text.translate(str.maketrans('', '', punctuation))

    # normalize spaces around periods to " . " → ". "
    text = re.sub(r'\s*\.\s*', '. ', text)

    # collapse multiple spaces and trim
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# Example
text = "1.  STABLE SMALL LEFT INTERNAL JUGULAR OPACITIES... 2.5 cm nodule; item 2. next. 3. Done."
print(clean_text(text))


# Example
text = """
 1.  INTERVAL PLACEMENT OF A RIGHT INTERNAL JUGULAR VENOUS SHEATH 
WITH THE DISTAL TIP IN THE PROXIMAL SUPERIOR VENA CAVA.  NO 
PNEUMOTHORAX.
 
 2.  STABLE POSITION OF NASOGASTRIC TUBE, FEEDING TUBE, TRACHEOSTOMY 
CANULA, LEFT INTERNAL JUGULAR CENTRAL VENOUS CATHETER, AND LEFT UPPER 
EXTREMITY PICC.  
 
 3.  NO SIGNIFICANT INTERVAL CHANGE IN HYPEREXPANDED LUNG VOLUMES, 
RIGHT BASILAR OPACITIES, SMALL BILATERAL PLEURAL EFFUSIONS, TENTING 
OF THE RIGHT HEMIDIAPHRAGM AND BIAPICAL PLEURAL THICKENING. 
 
 """
cleaned_text = clean_text(text)
print(cleaned_text)


stable small left internal jugular opacities. . . 2. 5 cm nodule item next. done.
interval placement of a right internal jugular venous sheath with the distal tip in the proximal superior vena cava. no pneumothorax. stable position of nasogastric tube feeding tube tracheostomy canula left internal jugular central venous catheter and left upper extremity picc. no significant interval change in hyperexpanded lung volumes right basilar opacities small bilateral pleural effusions tenting of the right hemidiaphragm and biapical pleural thickening.


In [22]:
encoded = tokenizer.encode(cleaned_text)
words = cleaned_text.split()
print("Number of words:", len(words), "Number of tokens:", len(encoded), "pad_id:", pad_id, "eos_id:", eos_id, "bos_id:", bos_id)
print("BOS token id:", tokenizer.bos_token_id, "EOS token id:", tokenizer.eos_token_id, "PAD token id:", tokenizer.pad_token_id)
print(encoded)
for token_id in encoded:    
    print(f"Token ID: {token_id}, Token: {tokenizer.decode([token_id])}")

Number of words: 65 Number of tokens: 112 pad_id: 50256 eos_id: 50256 bos_id: 50256
BOS token id: 50256 EOS token id: 50256 PAD token id: 50256
[50256, 3849, 2100, 13127, 286, 257, 826, 5387, 45808, 934, 8710, 516, 673, 776, 351, 262, 1233, 282, 8171, 287, 262, 14793, 4402, 9098, 410, 8107, 269, 4170, 13, 645, 29631, 849, 273, 897, 13, 8245, 2292, 286, 25221, 519, 459, 1173, 12403, 13017, 12403, 491, 4891, 455, 9145, 460, 4712, 1364, 5387, 45808, 934, 4318, 8710, 516, 3797, 43332, 290, 1364, 6727, 8963, 414, 8301, 66, 13, 645, 2383, 16654, 1487, 287, 20606, 21510, 79, 12249, 12317, 15343, 826, 1615, 1794, 1034, 330, 871, 1402, 24537, 3339, 1523, 914, 15880, 11105, 278, 286, 262, 826, 339, 13602, 72, 6570, 22562, 76, 290, 3182, 499, 605, 3339, 1523, 6546, 3101, 13, 50256]
Token ID: 50256, Token: 
Token ID: 3849, Token: inter
Token ID: 2100, Token: val
Token ID: 13127, Token:  placement
Token ID: 286, Token:  of
Token ID: 257, Token:  a
Token ID: 826, Token:  right
Token ID: 5387, Token: